# Создание столбцов на ходу
- Лена - разработчик
- Ваня - дизайнер
- Настя - редактор

In [1]:
import pandas as pd

In [10]:
def departments(row):
    dict_ = row['dicts']
    # print(dict_)
    
    if 'Лена' in dict_:
        row['разработчик'] = dict_['Лена']
        
    if 'Настя' in dict_:
        row['редактор'] = dict_['Настя']
    
    return row

In [11]:
df.head().apply(departments, axis=1)

,dicts,разработчик,редактор
0,"{'Лена': 1, 'Ваня': 2}",1.0,NaN
1,"{'Ваня': 3, 'Леша': 5}",NaN,NaN
2,{'Настя': 3},NaN,3.0


In [2]:
df = pd.DataFrame({'dicts': [{'Лена': 1, 'Ваня': 2}, {'Ваня': 3, 'Леша': 5}, {'Настя': 3}]})
df

,dicts
0,"{'Лена': 1, 'Ваня': 2}"
1,"{'Ваня': 3, 'Леша': 5}"
2,{'Настя': 3}


# Группировки и оконные функции

In [12]:
import pandas as pd

In [20]:
df = pd.DataFrame(
    {
        'city': ['Новосибирск', 'Новосибирск', 'Владивосток', 'Новосибирск', 'Владивосток', 'Владивосток', 'Владивосток', 'Владивосток', 'Новосибирск', 'Владивосток', None],
        'sales': [140, 135, 290, 95, 100, 50, 40, 120, 190, 200, 999],
    }
)
df

,city,sales
0,Новосибирск,140
1,Новосибирск,135
2,Владивосток,290
3,Новосибирск,95
4,Владивосток,100
5,Владивосток,50
6,Владивосток,40
7,Владивосток,120
8,Новосибирск,190
9,Владивосток,200


In [15]:
df['city'].unique()

array(['Новосибирск', 'Владивосток'], dtype=object)

In [16]:
df_novosib = df[df['city'] == 'Новосибирск']
df_novosib

,city,sales
0,Новосибирск,140
1,Новосибирск,135
3,Новосибирск,95
8,Новосибирск,190


In [17]:
df_novosib['sales'].sum()

560

In [21]:
df.groupby('city').sum()

,sales
city,
Владивосток,800
Новосибирск,560


Посчитать размах значений

In [23]:
df_novosib

,city,sales
0,Новосибирск,140
1,Новосибирск,135
3,Новосибирск,95
8,Новосибирск,190


In [24]:
max_sales = df_novosib['sales'].max()
max_sales

190

In [25]:
min_sales = df_novosib['sales'].min()
min_sales

95

In [26]:
max_sales - min_sales

95

In [27]:
def min_max(df):
    """На вход подается строки датафрейма с одинаковым ключом (городом)"""
    max_sales = df['sales'].max()
    min_sales = df['sales'].min()
    
    return max_sales - min_sales

In [28]:
df.groupby('city').apply(min_max)

city
Владивосток    250
Новосибирск     95
dtype: int64

In [22]:
df.groupby('city')

### Метод agg

In [29]:
df

,city,sales
0,Новосибирск,140
1,Новосибирск,135
2,Владивосток,290
3,Новосибирск,95
4,Владивосток,100
5,Владивосток,50
6,Владивосток,40
7,Владивосток,120
8,Новосибирск,190
9,Владивосток,200


In [35]:
df_novosib['sales'].max() - df_novosib['sales'].min()

95

In [36]:
def min_max_agg(column):
    """На вход подается столбец sales"""
    return column.max() - column.min()

In [38]:
df.groupby('city').agg({'sales': ['count', 'min', 'max', min_max_agg]})

sales                     
            count min  max min_max_agg
city                                  
Владивосток     6  40  290         250
Новосибирск     4  95  190          95

### Посчитать процент строки от ее региона
В SQL - over partition by

In [41]:
df['sales'].sum()

2359

In [40]:
df.sort_values('city')

,city,sales
2,Владивосток,290
4,Владивосток,100
5,Владивосток,50
6,Владивосток,40
7,Владивосток,120
9,Владивосток,200
0,Новосибирск,140
1,Новосибирск,135
3,Новосибирск,95
8,Новосибирск,190


In [45]:
a = [1, 2, 3]
b = a

b.append(10)
a

[1, 2, 3, 10]

In [46]:
df_vladik = df.copy()

df_vladik = df_vladik[df_vladik['city'] == 'Владивосток']
df_vladik

,city,sales
2,Владивосток,290
4,Владивосток,100
5,Владивосток,50
6,Владивосток,40
7,Владивосток,120
9,Владивосток,200


In [47]:
total_sum = df_vladik['sales'].sum()
total_sum

800

In [48]:
df_vladik['share'] = df_vladik['sales'] / total_sum
df_vladik

,city,sales,share
2,Владивосток,290,0.3625
4,Владивосток,100,0.1250
5,Владивосток,50,0.0625
6,Владивосток,40,0.0500
7,Владивосток,120,0.1500
9,Владивосток,200,0.2500


In [56]:
df_vladik.reset_index()

,index,city,sales,share
0,2,Владивосток,290,0.3625
1,4,Владивосток,100,0.1250
2,5,Владивосток,50,0.0625
3,6,Владивосток,40,0.0500
4,7,Владивосток,120,0.1500
5,9,Владивосток,200,0.2500


In [49]:
def share(df):
    total_sum = df['sales'].sum()
    df['share'] = df['sales'] / total_sum
    
    return df

In [51]:
df.groupby('city').apply(share).sort_values('city')

,city,sales,share
2,Владивосток,290,0.362500
4,Владивосток,100,0.125000
5,Владивосток,50,0.062500
6,Владивосток,40,0.050000
7,Владивосток,120,0.150000
9,Владивосток,200,0.250000
0,Новосибирск,140,0.250000
1,Новосибирск,135,0.241071
3,Новосибирск,95,0.169643
8,Новосибирск,190,0.339286


Если ключей больше

In [52]:
df = pd.DataFrame(
    {
        'region': ['СФО', 'СФО', 'ДВФО', 'СФО', 'ДВФО', 'ДВФО', 'ДВФО', 'ДВФО', 'СФО', 'ДВФО'],
        'city': ['Новосибирск', 'Томск', 'Владивосток', 'Новосибирск', 'Владивосток', 'Владивосток', 'Владивосток', 'Владивосток', 'Томск', 'Владивосток'],
        'sales': [140, 135, 290, 95, 100, 50, 40, 120, 190, 200],
    }
)
df

,region,city,sales
0,СФО,Новосибирск,140
1,СФО,Томск,135
2,ДВФО,Владивосток,290
3,СФО,Новосибирск,95
4,ДВФО,Владивосток,100
5,ДВФО,Владивосток,50
6,ДВФО,Владивосток,40
7,ДВФО,Владивосток,120
8,СФО,Томск,190
9,ДВФО,Владивосток,200


In [19]:
df.groupby('region').count()

,city,sales
region,,
ДВФО,6,6
СФО,4,4


In [53]:
df.groupby('region').apply(share).sort_values(['region', 'city'])

,region,city,sales,share
2,ДВФО,Владивосток,290,0.362500
4,ДВФО,Владивосток,100,0.125000
5,ДВФО,Владивосток,50,0.062500
6,ДВФО,Владивосток,40,0.050000
7,ДВФО,Владивосток,120,0.150000
9,ДВФО,Владивосток,200,0.250000
0,СФО,Новосибирск,140,0.250000
3,СФО,Новосибирск,95,0.169643
1,СФО,Томск,135,0.241071
8,СФО,Томск,190,0.339286


In [54]:
df.groupby(['region', 'city']).apply(share).sort_values(['region', 'city'])

,region,city,sales,share
2,ДВФО,Владивосток,290,0.362500
4,ДВФО,Владивосток,100,0.125000
5,ДВФО,Владивосток,50,0.062500
6,ДВФО,Владивосток,40,0.050000
7,ДВФО,Владивосток,120,0.150000
9,ДВФО,Владивосток,200,0.250000
0,СФО,Новосибирск,140,0.595745
3,СФО,Новосибирск,95,0.404255
1,СФО,Томск,135,0.415385
8,СФО,Томск,190,0.584615
